In [5]:
#import googlemaps
from datetime import datetime, timedelta
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
import random

# Setup
#gmaps = googlemaps.Client(key='YOUR_GOOGLE_MAPS_API_KEY')
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Functions
#def find_coffee_shops(location, radius=16093):  # 10 miles in meters
 #   places = gmaps.places_nearby(location=location, radius=radius, type='cafe')
 #   return places['results']

# Commented out the actual API calls for reviews
# def get_yelp_reviews(place_id):
#     headers = {'Authorization': 'Bearer YOUR_YELP_API_KEY'}
#     url = f'https://api.yelp.com/v3/businesses/{place_id}/reviews'
#     response = requests.get(url, headers=headers)
#     if response.status_code == 200:
#         return response.json()['reviews']
#     else:
#         print(f"Error: {response.status_code}")
#         return []

# def get_google_reviews(place_id):
#     place_details = gmaps.place(place_id=place_id, fields=['reviews'])
#     if 'result' in place_details and 'reviews' in place_details['result']:
#         return place_details['result']['reviews']
#     else:
#         return []

def filter_recent_reviews(reviews):
    six_months_ago = datetime.now() - timedelta(days=180)
    recent_reviews = [review for review in reviews if datetime.strptime(review['time_created'], '%Y-%m-%d %H:%M:%S') > six_months_ago]
    return recent_reviews

def filter_google_recent_reviews(reviews):
    six_months_ago = datetime.now() - timedelta(days=180)
    recent_reviews = [review for review in reviews if datetime.utcfromtimestamp(review['time']) > six_months_ago]
    return recent_reviews

def vectorize_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Function to generate random reviews
def generate_random_reviews():
    coffee_shop_names = ["Coffee Bliss", "Java Junction", "Brewed Awakenings", "The Grind", "Espresso Express", 
                         "Cuppa Joe", "Latte Love", "Mocha Magic", "Bean There", "Perk Up Cafe"]
    review_texts = [
        "This place is so cute and cozy! Perfect for a quick bite.",
        "A bit big and busy for my taste, but the coffee is great.",
        "Loved the ambiance, but there's usually a long queue.",
        "Great coffee but limited seating.",
        "Perfect spot for a quiet coffee break. Not too crowded.",
        "Nice place, but a bit noisy.",
        "Excellent coffee and friendly staff.",
        "Very spacious, but gets really busy during peak hours.",
        "Cute little shop with a nice variety of pastries.",
        "Good place for a quick coffee, but expect a wait during rush hours."
    ]
    reviews = []
    for i in range(20):
        name = random.choice(coffee_shop_names)
        text = random.choice(review_texts)
        time_created = (datetime.now() - timedelta(days=random.randint(0, 180))).strftime('%Y-%m-%d %H:%M:%S')
        reviews.append({"name": name, "text": text, "time_created": time_created})
    return reviews

# Aggregating the generated reviews
def aggregate_reviews():
    reviews = generate_random_reviews()
    return reviews

# Function to compare query with reviews
def compare_query_with_reviews(query, review_vectors):
    query_vector = vectorize_text(query)
    similarities = [(name, cosine_similarity(query_vector.detach().numpy(), vector.detach().numpy())[0][0]) for name, vector in review_vectors]
    return similarities

def generate_response(similarities):
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    top_choice = sorted_similarities[0]
    return f"We recommend {top_choice[0]} based on recent reviews."

# Example Usage
location = (40.6782, -73.9442)  # Example coordinates for Brooklyn
#coffee_shops = find_coffee_shops(location)
# all_reviews = aggregate_reviews(coffee_shops)
all_reviews = aggregate_reviews()  # Using the random reviews generator

review_vectors = [(review['name'], vectorize_text(review['text'])) for review in all_reviews]

query = "cute little coffee shop for bite to eat, 30 minutes before I have to start moving again for my next meeting"
similarities = compare_query_with_reviews(query, review_vectors)

response = generate_response(similarities)
print(response)

We recommend Espresso Express based on recent reviews.
